# Fully working scraper

In [1]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import random
from datetime import datetime
import requests

chromedriver_path = r'C:\Users\luisf\Desktop\MDS_thesis\scraper_gn\chromedriver-win64\chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [2]:
from datetime import datetime, timedelta

search_query = "Andrés Manuel López Obrador"
start_date = datetime.strptime("2019-12-01", "%Y-%m-%d")
end_date = datetime.strptime("2019-12-02","%Y-%m-%d")
max_pages = 1
daily_limit=1000 

current_date=start_date
count=0
current_page=1
url = f'https://www.google.com/search?q={search_query}+after:{start_date}+before:{end_date}&tbm=nws&sort=date' #the &gl=mx establishes the region
driver.get(url)


In [3]:
from bs4 import BeautifulSoup

In [4]:
#function for extracting information url by url 
import requests

def extract_full_text(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        soup = BeautifulSoup(response.text, 'html.parser')

        # Try common tags where article content is stored
        paragraphs = soup.find_all('p')  # Extract all paragraphs
        full_text = " ".join([p.get_text() for p in paragraphs])
        return full_text.strip() if full_text else None
    except Exception as e:
        print(f"Error extracting article text from {url}: {e}")
        return None


In [5]:
news_data = []

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    next_date = current_date + timedelta(days=1)
    formatted_next_date = next_date.strftime("%Y-%m-%d")
    
    # Update the URL for the daily range
    url = f'https://www.google.com/search?q={search_query}+after:{formatted_date}+before:{formatted_next_date}&tbm=nws&sort=date'
    print(f"Scraping URL for {formatted_date}: {url}")
    driver.get(url)

    count = 0
    current_page = 1

    while current_page <= max_pages and count < daily_limit:
        print(f"Scraping page {current_page} for {formatted_date}")
        
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf'))
            )
        except Exception as e:
            print(f"Timeout on {formatted_date}, page {current_page}: {e}")
            break

        news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
        if not news_results:
            print(f"No articles found on page {current_page}. Moving to next day.")
            break

        for news_div in news_results:
            if count >= daily_limit:
                break

            try:
                news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                news_title = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"]').text
                news_domain = news_div.find_element(By.CSS_SELECTOR, 'div').text
                news_date = news_div.find_element(By.CSS_SELECTOR, 'div.OSrXXb.rbYSKb.LfVVr > span').text
                
                # Extract full article text by visiting the URL
                full_text = extract_full_text(news_link)
                if not full_text:
                    print(f"Skipped article at {news_link} due to extraction error.")
                    continue
                
                # Append the full data to the list
                news_data.append([
                    news_link, news_domain, news_title, news_date, full_text
                ])
                count += 1
            except Exception as e:
                print(f"Error scraping an article: {e}")
                continue

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#pnnext'))
            )
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            time.sleep(random.uniform(2, 5))
            next_button.click()
            current_page += 1
            time.sleep(random.uniform(2, 4))
        except Exception as e:
            print(f"No more pages or error clicking 'Next': {e}")
            break

    current_date = next_date
    print(f"Finished scraping for {formatted_date}. Moving to the next day.")

Scraping URL for 2019-12-01: https://www.google.com/search?q=Andrés Manuel López Obrador+after:2019-12-01+before:2019-12-02&tbm=nws&sort=date
Scraping page 1 for 2019-12-01
Error extracting article text from https://www.washingtonpost.com/es/post-opinion/2019/12/01/un-anio-de-amlo-lo-bueno-lo-malo-y-lo-feo/: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=10)
Skipped article at https://www.washingtonpost.com/es/post-opinion/2019/12/01/un-anio-de-amlo-lo-bueno-lo-malo-y-lo-feo/ due to extraction error.
Error extracting article text from https://www.france24.com/es/20191201-inseguridad-estancamiento-econ%C3%B3mico-y-apoyos-sociales-marcan-el-primer-a%C3%B1o-de-amlo: 403 Client Error: Forbidden for url: https://www.france24.com/es/20191201-inseguridad-estancamiento-econ%C3%B3mico-y-apoyos-sociales-marcan-el-primer-a%C3%B1o-de-amlo
Skipped article at https://www.france24.com/es/20191201-inseguridad-estancamiento-econ%C3%B3mico-y-apoyos-sociales-m

In [9]:
columns = ['Link', 'Domain', 'Title', 'Date', 'Content']
news_df = pd.DataFrame(news_data, columns=columns)
print(news_df.head())


                                                Link  \
0  https://www.gob.mx/se/articulos/avances-de-los...   
1  https://www.bbc.com/mundo/noticias-america-lat...   
2  https://ibero.mx/prensa/opinion-primer-ano-de-...   
3  https://www.dw.com/es/m%C3%A9xico-l%C3%B3pez-o...   
4  https://www.infobae.com/america/mexico/2019/12...   

                                              Domain  \
0  gob.mx\nAvances de los Programas del Gobierno ...   
1  BBC\nInforme de AMLO: las decisiones de López ...   
2  IBERO\n#OPINIÓN Primer año de gobierno: la est...   
3  DW\nMéxico: López Obrador cumple primer año co...   
4  Infobae\nLópez Obrador a un año de gobierno: “...   

                                               Title        Date  \
0  Avances de los Programas del Gobierno del Pres...  1 dic 2019   
1  Informe de AMLO: las decisiones de López Obrad...  1 dic 2019   
2  #OPINIÓN Primer año de gobierno: la estrategia...  2 dic 2019   
3  México: López Obrador cumple primer año con pr...  

In [10]:
len(news_df)

270

In [11]:
news_df.to_csv("first_batch.csv")

# Previous code that partially worked 
Loop with pagination and visiting each webpage using the function above to extract the content of each news.
The following functions partially work. Up I am working on new ones. Jan 8 2025  

In [ ]:
#function for extracting information url by url 
def extract_full_text(article_url):
    try:
        # Send a request to the article URL
        response = requests.get(article_url, timeout=10)
        response.raise_for_status()  # Raise an error for bad HTTP responses
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the main article text (this varies by website)
        # Adjust the selector to target the main content
        article_body = soup.find_all('p')
        full_text = " ".join([p.text for p in article_body])
        return full_text
    except Exception as e:
        print(f"Error extracting article text from {article_url}: {e}")
        return None

In [15]:
news_data = []

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    next_date = current_date + timedelta(days=1)
    formatted_next_date = next_date.strftime("%Y-%m-%d")
    
    # Update the URL for the daily range
    url = f'https://www.google.com/search?q={search_query}+after:{formatted_date}+before:{formatted_next_date}&tbm=nws&sort=date'
    print(f"Scraping URL for {formatted_date}: {url}")
    driver.get(url)

    count = 0
    current_page = 1

    while current_page <= max_pages and count < daily_limit:
        print(f"Scraping page {current_page} for {formatted_date}")
        
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf'))
            )
        except Exception as e:
            print(f"Timeout on {formatted_date}, page {current_page}: {e}")
            break

        news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
        if not news_results:
            print(f"No articles found on page {current_page}. Moving to next day.")
            break

        for news_div in news_results:
            if count >= daily_limit:
                break

            try:
                news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                news_title = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"]').text
                news_domain = news_div.find_element(By.CSS_SELECTOR, 'div').text
                news_date = news_div.find_element(By.CSS_SELECTOR, 'div.OSrXXb.rbYSKb.LfVVr > span').text
                
                # Extract full article text by visiting the URL
                full_text = extract_full_text(news_link)
                if not full_text:
                    print(f"Skipped article at {news_link} due to extraction error.")
                    continue
                
                # Append the full data to the list
                news_data.append([
                    news_link, news_domain, news_title, news_date, full_text
                ])
                count += 1
            except Exception as e:
                print(f"Error scraping an article: {e}")
                continue

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#pnnext'))
            )
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            time.sleep(random.uniform(2, 5))
            next_button.click()
            current_page += 1
            time.sleep(random.uniform(2, 4))
        except Exception as e:
            print(f"No more pages or error clicking 'Next': {e}")
            break

    current_date = next_date
    print(f"Finished scraping for {formatted_date}. Moving to the next day.")

Scraping URL for 2018-12-03: https://www.google.com/search?q=mañanera amlo+after:2018-12-03+before:2018-12-04&tbm=nws&sort=date
Scraping page 1 for 2018-12-03
Error extracting article text from https://www.milenio.com/politica/la-mananera-de-que-hablo-amlo-en-su-primera-conferencia: name 'BeautifulSoup' is not defined
Skipped article at https://www.milenio.com/politica/la-mananera-de-que-hablo-amlo-en-su-primera-conferencia due to extraction error.
Error extracting article text from https://www.eleconomista.com.mx/politica/De-que-hablo-AMLO-en-su-primera-conferencia-de-prensa-matutina-20181203-0070.html: name 'BeautifulSoup' is not defined
Skipped article at https://www.eleconomista.com.mx/politica/De-que-hablo-AMLO-en-su-primera-conferencia-de-prensa-matutina-20181203-0070.html due to extraction error.
Error extracting article text from https://www.eluniversal.com.mx/nacion/el-dedito-el-gallito-y-el-beisbol-asi-eran-las-mananeras-de-amlo/: name 'BeautifulSoup' is not defined
Skipped a

KeyboardInterrupt: 

#loop with pagination, let's see if it works: 
(IT works)

In [13]:
while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    next_date = current_date + timedelta(days=1)
    formatted_next_date = next_date.strftime("%Y-%m-%d")

    # Update the URL for the daily range
    url = f'https://www.google.com/search?q={search_query}+after:{formatted_date}+before:{formatted_next_date}&tbm=nws&sort=date'
    print(f"Scraping URL for {formatted_date}: {url}")
    driver.get(url)

    # Pagination variables for the daily range
    count = 0
    current_page = 1

    # Pagination loop
    while current_page <= max_pages and count < daily_limit:
        print(f"Scraping page {current_page} for {formatted_date}")

        try:
            # Wait for articles to load
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf'))
            )
        except Exception as e:
            print(f"Timeout while waiting for articles on {formatted_date}, page {current_page}: {e}")
            break  # Exit pagination loop if no articles load

        news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
        if not news_results:
            print(f"No articles found on page {current_page}. Moving to next day.")
            break

        for news_div in news_results:
            if count >= daily_limit:
                break

            try:
                # Scrape article details
                news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                news_title = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"]').text
                news_domain = news_div.find_element(By.CSS_SELECTOR, 'div').text
                news_description = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"] + div').text
                news_date = news_div.find_element(By.CSS_SELECTOR, 'div.OSrXXb.rbYSKb.LfVVr > span').text

                news_data.append([news_link, news_domain, news_title, news_description, news_date])
                count += 1
            except Exception as e:
                print(f"Error scraping an article: {e}")
                continue

        # Handle pagination
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#pnnext'))
            )
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            time.sleep(random.uniform(2, 5))  # Mimic human behavior
            next_button.click()
            current_page += 1
            time.sleep(random.uniform(2, 4))  # Short wait for the next page to load
        except Exception as e:
            print(f"No more pages or error clicking 'Next': {e}")
            break  # Exit pagination loop if no next button

    # Increment to the next date
    current_date = next_date
    print(f"Finished scraping for {formatted_date}. Moving to the next day.")

Scraping URL for 2018-12-01: https://www.google.com/search?q=mañanera amlo+after:2018-12-01+before:2018-12-02&tbm=nws&sort=date
Scraping page 1 for 2018-12-01
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
Error scraping an article: name 'news_data' is not defined
No more pages or error clicking 'Next': Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF64966FB05+28789]
	(No symbol) [0x00007FF6495D86E0]
	(No symbol) [0x00007FF64947592A]
	(No symbol) [0x00007FF6494C930E]
	(No symbol) [0x00007FF6494C95FC]
	(No sy

KeyboardInterrupt: 

In [14]:
#Second iteration with pagination... fully functional. 

news_data = []

while current_page <= max_pages and count < limit:
    print(f"Scraping page {current_page}")
    
    try:
        # Wait for the news results to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf'))
        )
    except Exception as e:
        print(f"Timeout while waiting for news articles on page {current_page}: {e}")
        # Optional: Take a screenshot for debugging
        driver.save_screenshot(f'timeout_page_{current_page}.png')
        break  # Exit if articles do not load in time

    # Select all news article blocks by targeting the SoaBEf class
    news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
    
    if not news_results:
        print(f"No articles found on page {current_page}.")
        # Optional: Take a screenshot to inspect the page
        driver.save_screenshot(f'no_articles_page_{current_page}.png')
        break  # Exit if no articles are found

    for news_div in news_results:
        if count >= limit:
            print(f"Reached the limit of {limit} articles.")
            break  # Stop if we've reached the limit

        try:
            # Extract the link to the article
            news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')

            # Extract the title of the article
            news_title = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"]').text

            # Extract the source domain
            news_domain = news_div.find_element(By.CSS_SELECTOR, 'div').text

            # Extract the snippet description and the publication date
            news_description = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"] + div').text
            news_date = news_div.find_element(By.CSS_SELECTOR, 'div.OSrXXb.rbYSKb.LfVVr > span').text

            # Append the collected information to the list
            news_data.append([news_link, news_domain, news_title, news_description, news_date])
            count += 1  # Increment the count of news articles

        except Exception as e:
            print(f"Error scraping an article on page {current_page}: {e}")
            # Optional: Take a screenshot for debugging
            driver.save_screenshot(f'error_article_page_{current_page}.png')
            continue  # Skip to the next article

    # Check if we've reached the limit after scraping the current page
    if count >= limit:
        print(f"Reached the limit of {limit} articles.")
        break

    # Attempt to locate the "Next" button and click it using the general selector
    try:
        # Find the "Next" button using its general ID selector
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#pnnext'))
        )
        print("Found 'Next' button using the '#pnnext' selector.")
        
        # Scroll to the "Next" button to ensure it's in view
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        
        # Introduce a random sleep to mimic human behavior
        time.sleep(random.uniform(2, 5))
        
        # Capture the current URL before clicking "Next"
        current_url = driver.current_url
        print(f"Current URL before clicking 'Next': {current_url}")
        
        # Click the "Next" button
        next_button.click()
        print(f"Clicked 'Next' button on page {current_page}.")
        
        # Increment the current page counter
        current_page += 1
        
        # Optional: Add a short sleep to wait for the next page to load
        time.sleep(random.uniform(2, 4))
        
        # Capture the new URL after clicking "Next"
        new_url = driver.current_url
        print(f"Navigated to new URL: {new_url}")
        
        # Verify that the URL has changed to prevent looping
        if new_url == current_url:
            print("URL did not change after clicking 'Next'. Possible loop detected.")
            break  # Exit the loop to prevent infinite cycling
        else:
            print(f"Successfully navigated to page {current_page}.")

    except Exception as e:
        print(f"No more pages available or error clicking 'Next' on page {current_page}: {e}")
        # Optional: Take a screenshot to inspect the state when the error occurred
        #driver.save_screenshot(f'next_button_error_page_{current_page}.png')
        break  # Exit the loop if there's no next button or an error occurs

NameError: name 'limit' is not defined

In [5]:
print(news_data)
print(len(news_data))

[]
0


In [37]:
df = pd.DataFrame(news_data, columns=["Link", "Domain", "Title", "Description", "Date"])
print(df)
print(len(df))

Empty DataFrame
Columns: [Link, Domain, Title, Description, Date]
Index: []
0


In [25]:

# Save the results to a CSV file
csv_file_path = "daily_news_results_dec18.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(["Link", "Domain", "Title", "Description", "Date"])
    # Write the news data
    writer.writerows(news_data)

driver.quit()

print(f"Results saved to {csv_file_path}")

Results saved to daily_news_results_dec18.csv
